In [24]:
!pip install youtube-transcript-api
!pip install nltk
!pip install konlpy
!pip install gensim

import nltk
nltk.download('punkt')
nltk.download('stopwords')

import re
import logging

from youtube_transcript_api import YouTubeTranscriptApi
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from konlpy.tag import Okt, Kkma
from gensim.models.ldamodel import LdaModel
from gensim import corpora
from gensim.models.callbacks import CoherenceMetric, PerplexityMetric

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
import re
import logging
from youtube_transcript_api import YouTubeTranscriptApi
from konlpy.tag import Okt
from gensim.models.ldamodel import LdaModel
from gensim import corpora

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# 지금은 특정 링크를 직접 입력 웹에 적용시 사용자가 입력된 값이 전송되게 수정 예정
video_url = 'https://youtu.be/zztet70AYFA?si=WOEizYG8axGAWJPT'
print(video_url)

if 'youtu.be' in video_url:
    video_id = video_url.split('/')[-1].split('?')[0]
elif 'youtube.com/watch?v=' in video_url:
    video_id = video_url.split('v=')[-1].split('&')[0]
else:
    raise ValueError('Invalid YouTube URL')

print(video_id)

output = ''

try:
    transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['ko'])
    if not transcript:
        output = 'null'
    else:
        for x in transcript:
            sentence = x['text']
            output += f' {sentence}\n'
except Exception as e:
    print(f'오류 발생: {e}')
    output = 'null'

print(output)

output = re.sub(r'[^가-힣\s]', '', output)
output = output.strip()

stop_words = set(output.split())

okt = Okt()

def preprocess_korean_text(text):
    tokens = okt.pos(text)
    print('Tokens (Okt):', tokens)

    selected_tokens = [word for word, pos in tokens if pos.startswith('N') or pos == 'XR']
    print('Selected Tokens:', selected_tokens)

    selected_tokens = [token for token in selected_tokens if token not in stop_words]
    print('Filtered Tokens:', selected_tokens)

    return selected_tokens

preprocessed_tokens = preprocess_korean_text(output)

print('Preprocessed Tokens:', preprocessed_tokens)

https://youtu.be/zztet70AYFA?si=WOEizYG8axGAWJPT
zztet70AYFA
 한번
 빼고
 반갑다 야 얘 왜 뭐야 얼굴이 똑
 부러지네요
 [음악]
 이게 이평이 이게
 이잖아이 실명 와 실명 있으면 아
 근데 너무
 버섯
 오 버섯은
 맛있다
 어 해피 엘프티모 행복한
 이건 뭐야 라쿤 티모 오소리 티모
 오소리는 뭐
 똑같죠 드고 드거 티모가 뭐야 이건
 무슨
 티모야 악마 티모야
 얘는 뭐예요
 판다
 판단해 와
 우주비행사 야 우주비행사
 괜찮다 그래 이거지 난
 우주비행사 야 야야 뭐야 이거 뭐야
 이거 뭐야 이거 뭐야 뭐야 달
 정복이야기
 아 토끼 개킹 받긴
 한다 귀 움직이는 거 봐 이거 아
 근데 우주 비행사가 넘 사기네 나
 이거 우주비행사가 제일
 [음악]
 괜찮다 이거는는 똑같네
 오 어 이게 판다
 티모아 아까 건 악마티 모였네
 아 신 스킨이라고 와 이게
 악마아이 우디르탑 오랜만이네 우디르는
 그냥 죽여 놔야 돼 1레벨 라인 선시
 주도권 절대 주면 안돼 그냥 죽여야
 돼요 짜오 형 나 우디르 1레벨부터
 계속 딜교 나와서
 랩탑이나 첫 갱 탑 봐주면 땡큐요
 정도면 괜찮아 와주세요 보다는 봐주면
 땡큐로 가는게 더 보고 싶어 해
 정글러들이 이것도 랭의 꿀팁인데 나
 지금까지 그 좋게 얘기해서 와주는
 글들이 있거든 근데 대부분 안 오는
 정글도 좀 몇 번을 만나가지고 의견
 제시 오면은 잡는다 아니 오면 이득
 볼 만하다요 정도 근데 이판 좀 안
 중요한 데가 없네 미드도 탈론
 르블랑이 주도권 갖는 쪽이 좋고
 바텀도 라칸이라서 갱은 좋고
 가보자 거야
 두 번째
 합 다올 거 같은데 여기 근데 이거
 와드박 좋긴
 한데 생까지 30초
 남습니다 라인전지지 말기 우디르
 상대로이기는 거 못 본 듯 보여줄게요
 그러면 지금 당신 운 좋네 우디를
 상대로이기는 판을 지금 볼 수 있게
 돼서
 아으로 가면 어쩔 건데 우딜 경험치
 먹을 수
 있어 어쩔
 건데 경험치 먹을 수 있어 그렇게
 딜교 하면은

In [26]:
# LDA 모델
if preprocessed_tokens:
    dictionary = corpora.Dictionary([preprocessed_tokens])
    corpus = [dictionary.doc2bow(text) for text in [preprocessed_tokens]]

    num_topics = 5
    chunksize = 2000
    passes = 20
    iterations = 400
    eval_every = None

    model = LdaModel(
        corpus=corpus,
        id2word=dictionary,
        chunksize=chunksize,
        alpha='auto',
        eta='auto',
        iterations=iterations,
        num_topics=num_topics,
        passes=passes,
        eval_every=eval_every
    )

    for idx, topic in model.print_topics(-1):
        print(f'Topic {idx}: {topic}')
else:
    print('No valid tokens to build the LDA model.')

Topic 0: 0.010*"적" + 0.010*"상대로" + 0.010*"라카" + 0.010*"성장" + 0.010*"뒤" + 0.010*"땡큐" + 0.010*"주도" + 0.010*"뜰" + 0.010*"바" + 0.010*"마"
Topic 1: 0.028*"적" + 0.021*"상대로" + 0.021*"아군" + 0.014*"우드" + 0.014*"미드" + 0.014*"땡큐" + 0.014*"뜰" + 0.014*"라카" + 0.014*"마" + 0.014*"주도"
Topic 2: 0.010*"적" + 0.010*"냄새" + 0.010*"성장" + 0.010*"쟤" + 0.010*"앞" + 0.010*"말" + 0.010*"주도" + 0.010*"빼주" + 0.010*"지지" + 0.010*"애"
Topic 3: 0.010*"적" + 0.010*"아군" + 0.010*"애" + 0.010*"뒤" + 0.010*"땡큐" + 0.010*"뜰" + 0.010*"라카" + 0.010*"용" + 0.010*"말" + 0.010*"바"
Topic 4: 0.010*"적" + 0.010*"애" + 0.010*"쟤" + 0.010*"뜰" + 0.010*"라카" + 0.010*"우드" + 0.010*"용" + 0.010*"앞" + 0.010*"말" + 0.010*"바"


In [27]:
pip install transformers torch

In [28]:
from transformers import pipeline
from sklearn.cluster import KMeans
import numpy as np

In [29]:
# BERT 모델
sentiment_analyzer = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')

text_for_sentiment = " ".join(preprocessed_tokens)
sentiment_result = sentiment_analyzer(text_for_sentiment)

print("Sentiment Analysis Result:", sentiment_result)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Sentiment Analysis Result: [{'label': 'NEGATIVE', 'score': 0.527480959892273}]


In [30]:
embeddings = np.array(embeddings)

num_topics = 5
kmeans = KMeans(n_clusters=num_topics, random_state=42)
kmeans.fit(embeddings)

labels = kmeans.labels_
print("Cluster Labels:", labels)

clustered_documents = {i: [] for i in range(num_topics)}
for label, token in zip(labels, preprocessed_tokens):
    clustered_documents[label].append(token)

for cluster, docs in clustered_documents.items():
    print(f"Cluster {cluster}: {docs}")


Cluster Labels: [1 1 2 0 1 1 1 2 1 3 1 3 3 1 2 1 1 0 3 2 1 3 3 0 3 0 1 1 3 3 1 4 4 3 1 3 1
 3 3 3 1 1 1 1 4 3 4 1 4 0 2 1 0 3 1 3 3 3 1 1 3 3 1 1 3 3 3 4 1 4 4 1 1 1
 0 3 1 3 3 4 1 1 3 1 3 1 4 1 3 3 0 1 1 3 3 4 3 1 4 1 0 4 3 3 4 4 1 1 4 0 3
 1 3 3 4 4 3 2 3 4 3 1 1 0 1 1 1 0 1 3 1]
Cluster 0: ['드', '러', '만', '데', '마', '무', '로', '마', '바', '임', '바', '기']
Cluster 1: ['얼굴', '이평', '판단', '정복', '이야기', '사기', '스킨', '주도', '땡큐', '땡큐', '얘기', '미드', '주도', '지지', '감당', '아군', '확인', '아군', '계획', '성대', '견제', '아군', '우드', '사스', '우드', '다이브', '냄새', '시간', '포탑', '카정', '미드', '성장', '리안', '노궁', '대안', '라카', '라카', '뜰', '뜰', '블랑', '초식', '문제', '추천', '생각', '공격', '오대', '지지', '다행', '빡세', '성장', '용이']
Cluster 2: ['엘프', '비행사', '랩탑', '꿀팁', '기자', '삭발']
Cluster 3: ['티', '아이', '만이', '랭', '글', '번', '판', '쪽', '생', '판', '길이', '적', '비', '뒤', '글', '득', '데이', '내', '적', '적', '적', '앞', '앞', '두루', '드루', '탈', '내', '절', '세', '용', '용', '망', '잡', '뒤', '블', '살', '애', '키', '애', '지면', '말', '렌', '말']
Cluster 4: ['상대로', '상대로', '예', '요음', '가줄', '